In [84]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

from pymystem3 import Mystem

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



In [85]:

def Lemmatize(src: list):
    ''' Лемматизирует переданный датасет
    '''
    text_nomalized = ' '.join(src).lower() 

    m = Mystem()
    lemmas = m.lemmatize(text_nomalized)
    
    return lemmas

In [86]:
data = pd.read_csv('dataframe.csv')
data = data.head(5)
print(data)
#data = data.head(10)
#«Обучающая выборка» — это набор данных, который используется для разработки модели машинного обучения.

#«Валидационная выборка» — это набор данных, который используется в процессе разработки модели машинного обучения для подбора оптимального набора гиперпараметров.

#«Тестовая выборка» — это набор данных, который не используется непосредственно в процессе обучения модели или для подбора гиперпараметров, 
# однако позволяет протестировать модель и является контрольным.

   mark                                    text_of_comment
0     1  Сразу отвечаю на вопрос, почему я решила прочи...
1     1  Эта сказка хоть и сказка,но учит всех людей то...
2     1  Несомненно видно мастерство автора, который с ...
3     1  Эта сказка хоть и сказка,но учит всех людей то...
4     1  Несомненно видно мастерство автора, который с ...


In [87]:
data.dropna(inplace=True)
remove_non_alphabets = lambda x: re.sub(r'[^а-яА-Я]',' ',str(x))

tokenize = lambda x: word_tokenize(x, language = "russian")

ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

print('remove_non_alphabets')
data['text_of_comment'] = data['text_of_comment'].apply(remove_non_alphabets)

print('tokenize')
data['text_of_comment'] = data['text_of_comment'].apply(tokenize) # [ word_tokenize(row) for row in data['email']]

print('stem')
data['text_of_comment'] = data['text_of_comment'].apply(stem)

print('Lemmatize')
c = 0
for words_list in data['text_of_comment']:
    words_list = Lemmatize(words_list)
    print(c)
    c+=1

print('complete')
data['text_of_comment'] = data['text_of_comment'].apply(lambda x: ' '.join(x))

data.dropna(inplace=True)

print(data)


remove_non_alphabets
tokenize
stem
Lemmatize
0
1
2
3
4
complete
   mark                                    text_of_comment
0     1  сразу отвечаю на вопрос почему я решила прочит...
1     1  эта сказка хоть и сказка но учит всех людей то...
2     1  несомненно видно мастерство автора который с п...
3     1  эта сказка хоть и сказка но учит всех людей то...
4     1  несомненно видно мастерство автора который с п...


In [89]:
#max_words = 10000
stopWords = stopwords.words('russian')

vectorizer = CountVectorizer(stop_words=stopWords)
sparse_matrix = vectorizer.fit_transform(data['text_of_comment']).toarray()
print(sparse_matrix)
sparse_matrix.shape

[[0 0 0 1 1 0 1 1 0 0 0 0 1 2 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 2 1 0 1 0
  0 0 1 0 0 0 0 1 4 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 2 0 0 1 0 0
  0 0 1 0 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 2 1 1 0 1 0 1 0 0 0
  0 0 0 0 3 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0]
 [1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 2 1 0 1 1 0 1 1 0 0 2 0 1
  1 1 0 0 1 1 2 0 0 1 1 0 2 1 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1
  1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 1
  0 1 1 2 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 1 1 1 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1

(5, 143)

In [91]:
x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(data['mark']), test_size = 0.8)

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x